In [ ]:
import pandas as pd

from meridian import constants
from meridian.data import data_frame_input_data_builder as data_builder
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution

from mmm_eval import (
    MeridianConfig, MeridianInputDataBuilderSchema, run_evaluation)

import tensorflow_probability as tfp


## Load your data

The data below comes from the Meridian repository and gives a great example of how the data should be formatted.

In [2]:
df = pd.read_excel(
    'https://github.com/google/meridian/raw/main/meridian/data/simulated_data/xlsx/geo_media.xlsx',
    engine='openpyxl',
)
df.head()

,Unnamed: 0,geo,time,Channel0_impression,Channel1_impression,Channel2_impression,Channel3_impression,Channel4_impression,Channel5_impression,Competitor_Sales,...,GQV,Channel0_spend,Channel1_spend,Channel2_spend,Channel3_spend,Channel4_spend,Channel5_spend,conversions,revenue_per_conversion,population
0,0,Geo0,2021-01-25,79241,259401,0,0,450937,95311,-1.338765,...,-0.259983,564.373108,1902.115723,0.000000,0.0,3514.080078,742.654846,1957658.500,0.020055,136670.9375
1,1,Geo0,2021-02-01,167418,326184,174129,0,490676,228607,0.893645,...,-0.489272,1192.390503,2391.816895,1678.817993,0.0,3823.759766,1781.285522,2058891.125,0.020103,136670.9375
2,2,Geo0,2021-02-08,0,197565,230170,0,393618,184061,-0.284549,...,0.683819,0.000000,1448.689453,2219.122314,0.0,3067.402344,1434.187012,1903555.125,0.019929,136670.9375
3,3,Geo0,2021-02-15,0,140990,66643,0,326034,201729,-1.034740,...,1.289055,0.000000,1033.840576,642.520569,0.0,2540.730957,1571.854492,2503275.500,0.019987,136670.9375
4,4,Geo0,2021-02-22,0,399116,164991,0,381982,153973,-0.319276,...,0.227739,0.000000,2926.607178,1590.716431,0.0,2976.724854,1199.744019,3489248.000,0.020000,136670.9375


A meridian model object doesnt accept a dataframe, it instead accepts an instance of its internal data_builder object.
We create this object below providing the required arguments (media channels, spend columns, kpi column etc)

In [3]:
channels = ["Channel0", "Channel1", "Channel2", "Channel3", "Channel4", "Channel5"]
control_cols = ["GQV", "Discount", "Competitor_Sales"]
media_cols = [f"{channel}_impression" for channel in channels]
media_spend_cols = [f"{channel}_spend" for channel in channels]

builder = (
    data_builder.DataFrameInputDataBuilder(kpi_type='non_revenue')
        .with_kpi(df, kpi_col="conversions")
        .with_revenue_per_kpi(df, revenue_per_kpi_col="revenue_per_conversion")
        .with_population(df)
        .with_controls(df, control_cols=control_cols)
)
builder = builder.with_media(
    df,
    media_cols=media_cols,
    media_spend_cols=media_spend_cols,
    media_channels=channels,
)

data = builder.build()
data

InputData(kpi=<xarray.DataArray 'kpi' (geo: 40, time: 156)> Size: 50kB
array([[ 1957658.5  ,  2058891.125,  1903555.125, ...,  2028144.5  ,
         2106282.   ,  2302750.75 ],
       [ 2459645.   ,  1892454.25 ,  7346417.   , ...,  2514223.25 ,
         5834204.   ,   551180.5  ],
       [ 9007940.   , 13928220.   , 15245770.   , ..., 12689826.   ,
         7348204.   , 16124357.   ],
       ...,
       [ 3990133.25 ,  6108879.5  ,  5789613.5  , ...,  6927744.   ,
         9038672.   ,  7755718.5  ],
       [ 2634550.   ,  5750421.5  ,  5156670.   , ...,  5085422.   ,
         6665999.   ,  5173200.   ],
       [23258282.   , 11010188.   , 16292038.   , ..., 25776564.   ,
        15508774.   , 19470110.   ]])
Coordinates:
  * time     (time) <U10 6kB '2021-01-25' '2021-02-01' ... '2024-01-15'
  * geo      (geo) <U5 800B 'Geo0' 'Geo1' 'Geo2' ... 'Geo37' 'Geo38' 'Geo39', kpi_type='non_revenue', population=<xarray.DataArray 'population' (geo: 40)> Size: 320B
array([136670.9375  , 199816.

## Define a Meridian MMM

A meridian model requires one arguments - input data (in the form of a data builder object)

As an example, we also include an identical set of priors for all media channels (based on a Lognormal(0.2, 0.9) distribution).

In [4]:
roi_mu = 0.2     # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)
mmm = model.Meridian(input_data=data, model_spec=model_spec)

# Preprocess the data for the evaluation suite

One requirement of the evaluation suite is that it expects revenue to run its tests, not revenue per KPI.

We apply this change to our dataframe below

In [5]:
data_preproc = df.copy()
data_preproc["revenue"] = data_preproc["revenue_per_conversion"]*data_preproc["conversions"]

# restrict to only two geos
data_preproc = data_preproc[data_preproc["geo"].isin(["Geo0"])]

# restrict to only post-2023
data_preproc = data_preproc[pd.to_datetime(data_preproc["time"]) > pd.Timestamp("2023-01-01")]
data_preproc = data_preproc[pd.to_datetime(data_preproc["time"]) < pd.Timestamp("2023-11-01")]


data_preproc.head()

,Unnamed: 0,geo,time,Channel0_impression,Channel1_impression,Channel2_impression,Channel3_impression,Channel4_impression,Channel5_impression,Competitor_Sales,...,Channel0_spend,Channel1_spend,Channel2_spend,Channel3_spend,Channel4_spend,Channel5_spend,conversions,revenue_per_conversion,population,revenue
101,101,Geo0,2023-01-02,0,73792,21261,0,488327,188366,0.181207,...,0.000000,541.096313,204.982224,0.0,3805.454590,1467.731201,2054979.125,0.019946,136670.9375,40988.019528
102,102,Geo0,2023-01-09,314125,244492,158008,0,409000,0,-0.179270,...,2237.272461,1792.792114,1523.391724,0.0,3187.271729,0.000000,1499235.750,0.020027,136670.9375,30025.077764
103,103,Geo0,2023-01-16,0,252992,0,0,280268,184126,-0.507176,...,0.000000,1855.120239,0.000000,0.0,2184.083740,1434.693481,1286753.875,0.020062,136670.9375,25814.551284
104,104,Geo0,2023-01-23,0,0,0,0,95579,13865,-1.684848,...,0.000000,0.000000,0.000000,0.0,744.831848,108.034851,3257323.250,0.019879,136670.9375,64752.232663
105,105,Geo0,2023-01-30,0,0,0,0,0,0,-0.444570,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,3082807.250,0.019957,136670.9375,61523.114631


# Create an `mmm-eval` config 

The `mmm-eval` package can work with a variety of frameworks. It does this by allowing you to create a config from some simple arguments. These slightly differ between frameworks, but for meridian this involves just
1. the meridian model object
2. the date column
3. the media channels
4. the media spend column names
5. the response column

In [6]:
# Create an instance of the mmm-eval Meridian Config
input_data_builder_config = MeridianInputDataBuilderSchema(
    date_column="time",
    media_channels=channels,
    channel_spend_columns=media_spend_cols,
    channel_impressions_columns=media_cols,
    response_column="conversions",
    control_columns=control_cols, #Optional but provided for example
)
input_data_builder_config

MeridianInputDataBuilderSchema(date_column='time', media_channels=['Channel0', 'Channel1', 'Channel2', 'Channel3', 'Channel4', 'Channel5'], channel_spend_columns=['Channel0_spend', 'Channel1_spend', 'Channel2_spend', 'Channel3_spend', 'Channel4_spend', 'Channel5_spend'], channel_impressions_columns=['Channel0_impression', 'Channel1_impression', 'Channel2_impression', 'Channel3_impression', 'Channel4_impression', 'Channel5_impression'], channel_reach_columns=None, channel_frequency_columns=None, organic_media_columns=None, organic_media_channels=None, non_media_treatment_columns=None, response_column='conversions', control_columns=['GQV', 'Discount', 'Competitor_Sales'])

Crate the full `mmm-eval` meridian config

In [7]:
# specify a larger number of samples if you want quality results
sample_posterior_kwargs = dict(n_chains=1, n_adapt=10, n_burnin=10, n_keep=10) #Example of setting the samples and chains if we dont want to use the defaults
config = MeridianConfig.from_model_object(mmm, 
                                          input_data_builder_config=input_data_builder_config,
                                          revenue_column="revenue", 
                                          sample_posterior_kwargs=sample_posterior_kwargs
                                          )
config

MeridianConfig(revenue_column='revenue', response_column='conversions', input_data_builder_config=MeridianInputDataBuilderSchema(date_column='time', media_channels=['Channel0', 'Channel1', 'Channel2', 'Channel3', 'Channel4', 'Channel5'], channel_spend_columns=['Channel0_spend', 'Channel1_spend', 'Channel2_spend', 'Channel3_spend', 'Channel4_spend', 'Channel5_spend'], channel_impressions_columns=['Channel0_impression', 'Channel1_impression', 'Channel2_impression', 'Channel3_impression', 'Channel4_impression', 'Channel5_impression'], channel_reach_columns=None, channel_frequency_columns=None, organic_media_columns=None, organic_media_channels=None, non_media_treatment_columns=None, response_column='conversions', control_columns=['GQV', 'Discount', 'Competitor_Sales']), model_spec_config=MeridianModelSpecSchema(prior=PriorDistribution(knot_values=<tfp.distributions.Normal 'knot_values' batch_shape=[] event_shape=[] dtype=float32>, tau_g_excl_baseline=<tfp.distributions.Normal 'tau_g_excl_

# Run the mmm_eval suite

The evaluation suite requires 3 arguments
1. The framework name
2. The `mmm-eval` version of the config for that framework
3. The dataframe to run the tests on

We can also include the tests to run. If they are not included, it runs all tests by default

In [8]:
# Run the evaluation suite!
result = run_evaluation(framework="meridian", config=config, data=data_preproc, test_names=["holdout_accuracy"])
result

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,mape,mape,33.966792,False,holdout_accuracy,2025-08-02T15:24:01.309671
1,smape,smape,44.510339,False,holdout_accuracy,2025-08-02T15:24:01.309671
2,r_squared,r_squared,-1.229455,False,holdout_accuracy,2025-08-02T15:24:01.309671


In [16]:
# Run the evaluation suite!
result = run_evaluation(framework="meridian", config=config, data=data_preproc, test_names=["cross_validation"])
result

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,mean_mape,mean_mape,30.036020,False,cross_validation,2025-08-03T14:45:36.873890
1,std_mape,std_mape,5.256887,False,cross_validation,2025-08-03T14:45:36.873890
2,mean_smape,mean_smape,35.014834,False,cross_validation,2025-08-03T14:45:36.873890
3,std_smape,std_smape,11.922026,False,cross_validation,2025-08-03T14:45:36.873890
4,mean_r_squared,mean_r_squared,-0.196312,False,cross_validation,2025-08-03T14:45:36.873890


In [8]:
# Run the evaluation suite!
result = run_evaluation(framework="meridian", config=config, data=data_preproc, test_names=["placebo"])
result

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,shuffled_channel_roi,shuffled_channel_roi_Channel0_shuffled,-7.633817,False,placebo,2025-08-02T18:08:18.885909


In [9]:
# Run the evaluation suite!
result = run_evaluation(framework="meridian", config=config, data=data_preproc, test_names=["in_sample_accuracy"])
result

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,mape,mape,29.824497,False,in_sample_accuracy,2025-08-02T18:09:39.772036
1,smape,smape,26.875219,False,in_sample_accuracy,2025-08-02T18:09:39.772036
2,r_squared,r_squared,0.354487,False,in_sample_accuracy,2025-08-02T18:09:39.772036


In [8]:
# Run the evaluation suite!
result = run_evaluation(framework="meridian", config=config, data=data_preproc, test_names=["perturbation"])
result

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,percentage_change,percentage_change_Channel0,95.060646,False,perturbation,2025-08-03T14:28:58.206225
1,percentage_change,percentage_change_Channel1,238.593544,False,perturbation,2025-08-03T14:28:58.206225
2,percentage_change,percentage_change_Channel2,1055.118345,False,perturbation,2025-08-03T14:28:58.206225
3,percentage_change,percentage_change_Channel3,48.323219,False,perturbation,2025-08-03T14:28:58.206225
4,percentage_change,percentage_change_Channel4,116.035639,False,perturbation,2025-08-03T14:28:58.206225
5,percentage_change,percentage_change_Channel5,148.851493,False,perturbation,2025-08-03T14:28:58.206225


In [8]:
# Run the evaluation suite!
result = run_evaluation(framework="meridian", config=config, data=data_preproc, test_names=["refresh_stability"])
result

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,mean_percentage_change,mean_percentage_change_Channel0,99.155874,False,refresh_stability,2025-08-03T15:01:18.482792
1,mean_percentage_change,mean_percentage_change_Channel1,417.025152,False,refresh_stability,2025-08-03T15:01:18.482792
2,mean_percentage_change,mean_percentage_change_Channel2,312.913243,False,refresh_stability,2025-08-03T15:01:18.482792
3,mean_percentage_change,mean_percentage_change_Channel3,223.468202,False,refresh_stability,2025-08-03T15:01:18.482792
4,mean_percentage_change,mean_percentage_change_Channel4,141.362042,False,refresh_stability,2025-08-03T15:01:18.482792
5,mean_percentage_change,mean_percentage_change_Channel5,114.704491,False,refresh_stability,2025-08-03T15:01:18.482792
6,std_percentage_change,std_percentage_change_Channel0,37.458950,False,refresh_stability,2025-08-03T15:01:18.482792
7,std_percentage_change,std_percentage_change_Channel1,638.996071,False,refresh_stability,2025-08-03T15:01:18.482792
8,std_percentage_change,std_percentage_change_Channel2,313.987235,False,refresh_stability,2025-08-03T15:01:18.482792
9,std_percentage_change,std_percentage_change_Channel3,272.016323,False,refresh_stability,2025-08-03T15:01:18.482792


In [9]:
# Run the evaluation suite!
result = run_evaluation(framework="meridian", config=config, data=data_preproc)
result

: 